In [7]:
import time
import asyncio
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from concurrent.futures import ThreadPoolExecutor

import yarl # not necessary?


# ANSI colors
c = (
    "\033[0m",   # End of color
    "\033[36m",  # Cyan
    "\033[91m",  # Red
    "\033[35m",  # Magenta
)



In [3]:
# # TODO: https://medium.com/@sakthiveltvt.thangaraj/introduction-to-nest-asyncio-for-python-developers-afd7bed44768
# # TODO: https://www.bing.com/search?pglt=161&q=asynchio+nesting

# global results
# results = {}
# global job_ad_ids
# job_ad_ids = []

# # Function to scrape one page using Selenium
# async def scrape_with_selenium(url):
#     options = Options()
#     # options.add_argument("--headless")  # Run in headless mode
#     driver = webdriver.Chrome(options=options)
#     try:
#         driver.get(url)
#         content = driver.page_source
#         print(f"Scraped {url}")
#         # print(content)
#         article = driver.find_elements(By.TAG_NAME, "article")
#         # print('article: ', article)
#         with ThreadPoolExecutor(max_workers=30) as executor:
#             tasks = [async_populate_results(element, executor) for element in article]
#             results = await asyncio.gather(*tasks)
#             # executor.map(populate_results, article)
#         return content
#     finally:
#         driver.quit()

# # Async wrapper to run the blocking Selenium code
# # async def async_populate_results(element, executor):
# #     loop = asyncio.get_running_loop()
# #     return await loop.run_in_executor(executor, populate_results, element)

# async def populate_results(article):
#     print(article)
#     print('populating results...')
#     # job_ad_id = article.get_attribute("data-job-id")
#     # job_ad_ids.append(job_ad_id)
#     # results[job_ad_id] = {}
#     # elements = article.find_elements(By.CSS_SELECTOR, '[data-automation]')
#     # for el in elements:
#     #     results[job_ad_id][el.get_attribute("data-automation").replace('-','_')] = el.text
#     # # results.append([f'{element.get_attribute("data-automation").replace("-","_")}:{element.text}' for element in elements])

# # Async wrapper to run the blocking Selenium code
# async def async_scrape(url, executor):
#     loop = asyncio.get_running_loop()
#     return await loop.run_in_executor(executor, scrape_with_selenium, url)

# # Main async function to scrape multiple URLs
# async def main(urls):
#     results = []
#     with ThreadPoolExecutor(max_workers=30) as executor:
#         tasks = [async_scrape(url, executor) for url in urls]
#         print(len(tasks))
#         results = await asyncio.gather(*tasks)
#     print(f"\nScraped {len(results)} pages.")

# # Example usage
# if __name__ == "__main__":
#     start_time = time.perf_counter()
#     urls = [
#     f'https://www.seek.com.au/Data-Engineer-jobs/in-Melbourne-VIC-3000?page={x+1}' for x in range(2)
#     ]
#     await main(urls)
#     print('run time = ', time.perf_counter() - start_time)

<h3 style="color: #F34222;">VERSION 2</h3><p><h4 style="color: #F3FF22;">async/await</h4></p>

In [22]:
a, b = 1,2

In [43]:
# TODO: https://medium.com/@sakthiveltvt.thangaraj/introduction-to-nest-asyncio-for-python-developers-afd7bed44768
# TODO: https://www.bing.com/search?pglt=161&q=asynchio+nesting

import time
import asyncio
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from concurrent.futures import ThreadPoolExecutor

# articles = []
# global job_ads
job_ads = {}
# job_ids = []


def parse_article(el, job_ad_id):
    job_ads[job_ad_id][(el.get_attribute("data-automation")).replace('-','_')] = el.text 

def parse_articles(article):
    start_time_outer = time.perf_counter()
    try:
        print(f'parsing articles')
        job_id = article.get_attribute("data-job-id")
        print(f'jid: {c[2]}{job_id}{c[0]}')
        job_ads[job_id], start_time_inner, elements = {}, time.perf_counter(), article.find_elements(By.CSS_SELECTOR, '[data-automation]')
        print(f'{c[1]}find data elements time:{time.perf_counter() - start_time_inner}s | startime:{start_time_inner}{c[0]}')
        # with ThreadPoolExecutor(max_workers=64) as executor:
        #     result = executor.map(parse_article, elements, [job_id for i in elements])
        # executor.shutdown(wait=True) # Shutdown the executor
        for el in elements: #data-automation data-testid
            job_ads[job_id][(el.get_attribute("data-automation")).replace('-','_')] = el.text
    except Exception as e:
        print('Error parse_articles(): ', e)
    finally:
        print(f'{c[3]}parse article time = {time.perf_counter() - start_time_outer}, startime: {start_time_outer}{c[0]}')

# Function to scrape one page using Selenium
def scrape_with_selenium(url):
    try:
        # options = Options()
        options = webdriver.ChromeOptions()
        options.page_load_strategy = 'none'
        driver = webdriver.Chrome(options=options)
        driver.get(url)
        print(f"Scraped {url}")
        start_time = time.perf_counter()
        articles = WebDriverWait(driver, 22).until(
            EC.presence_of_all_elements_located((By.TAG_NAME, 'article'))
        )
        print(f'articles collected... {c[1]}collection time: {time.perf_counter() - start_time}s | start time: {start_time}{c[0]}')
        start_time = time.perf_counter()
        (print('stopping page load...'), driver.execute_script("window.stop();"))
        try:
            # with ThreadPoolExecutor(max_workers=64) as executor:
            #     print('EXECUTOR🪓🪓🪓')
            #     result = executor.map(parse_articles, articles) #articles[:5])
            # executor.shutdown(wait=True) # Shutdown the executor
            jobs = driver.execute_script("""
                let articles = Array.from(document.getElementsByTagName('article'))
                let jobs = {}
                articles.map(article => {
                	let jobId = article.dataset.jobId
                	jobs[jobId] = {}
                	let elements = Array.from(article.querySelectorAll('[data-automation]'))
                	return elements.map(el => {
                		let dataKey = el.dataset.automation
                		jobs[jobId][dataKey] = el.innerText
                	})
                })
                return jobs
            """)
            job_ads.update(jobs)
        except Exception as e:
            print('Error:', e)
        finally:
            driver.quit()
        print('parse end time = ', time.perf_counter())
        print('run time (parsing article) = ', time.perf_counter() - start_time)
        
    except Exception as e:
        print('Error: scrape_with_selenium()', e)
    finally:
        # driver.quit()
        return articles

# Async wrapper to run the blocking Selenium code
async def async_scrape(url, executor):
    try:
        loop = asyncio.get_running_loop()
        return await loop.run_in_executor(executor, scrape_with_selenium, url)
    except Exception as e:
        print('Error:', e)
    
async def populate_results(url, executor):
    try:
        article = await async_scrape(url, executor)
        # articles.append(article)
    except Exception as e:
        print('Error:', e)

# Main async function to scrape multiple URLs
async def main(urls):
    try:
        results = []
        with ThreadPoolExecutor(max_workers=128) as executor:
            tasks = [populate_results(url, executor) for url in urls]
            results = await asyncio.gather(*tasks)
        print(f"\nScraped {len(results)} pages.")
    except Exception as e:
        print('Error:', e)


# Example usage
if __name__ == "__main__":
    try:
        start_time = time.perf_counter()
        urls = [
        f'https://www.seek.com.au/Data-Engineer-jobs/in-Melbourne-VIC-3000?page={x+1}' for x in range(10)
        ]
        await main(urls)
        print('run time = ', time.perf_counter() - start_time)
    except Exception as e:
        print('Error:', e)

Scraped https://www.seek.com.au/Data-Engineer-jobs/in-Melbourne-VIC-3000?page=1
Scraped https://www.seek.com.au/Data-Engineer-jobs/in-Melbourne-VIC-3000?page=3
Scraped https://www.seek.com.au/Data-Engineer-jobs/in-Melbourne-VIC-3000?page=9
Scraped https://www.seek.com.au/Data-Engineer-jobs/in-Melbourne-VIC-3000?page=10
Scraped https://www.seek.com.au/Data-Engineer-jobs/in-Melbourne-VIC-3000?page=8
Scraped https://www.seek.com.au/Data-Engineer-jobs/in-Melbourne-VIC-3000?page=2
Scraped https://www.seek.com.au/Data-Engineer-jobs/in-Melbourne-VIC-3000?page=4
Scraped https://www.seek.com.au/Data-Engineer-jobs/in-Melbourne-VIC-3000?page=6
Scraped https://www.seek.com.au/Data-Engineer-jobs/in-Melbourne-VIC-3000?page=7
Scraped https://www.seek.com.au/Data-Engineer-jobs/in-Melbourne-VIC-3000?page=5
articles collected... collection time: 1.0660961499997939s | start time: 17128.761386654
stopping page load...
articles collected... collection time: 1.189876885000558s | start time: 17128.773374657


In [55]:
# for article in articles[0]:
#     parse_articles(article, 'xyz')

In [29]:
# job_ads = {}

In [42]:
len(job_ads)

187

In [86]:
import itertools
d = job_ads
dict(itertools.islice(d.items(), 1))

{'85163127': {'job_list_view_job_link': '',
  'job_list_item_link_overlay': '',
  'company_logo_container': '',
  'company_logo': '',
  'jobTitle': 'AI ENGINEER',
  'jobCompany': 'Opus Recruitment Solutions',
  'jobCardLocation': 'Melbourne VIC',
  'jobLocation': 'Melbourne VIC',
  'jobSalary': 'AUD 900 - 1150 per day',
  'jobShortDescription': 'AI ENGINEER | 6 MONTH CONTRACT |MELBOURNE | HYBRID WORKING',
  'jobSubClassification': '',
  'jobClassification': '',
  'signed_out_save_job': ''}}

<h3 style="color: #F34222;">VERSION 3</h3><p><h4 style="color: #F3FF22;">using a queue</h4></p>

In [ ]:
# https://www.bing.com/search?q=asyncio+queue+python+epxlained&cvid=d0b1708a716e47ba94a1cc3cc9f1a517&gs_lcrp=EgRlZGdlKgYIABBFGDkyBggAEEUYOTIGCAEQABhAMgYIAhAAGEAyBggDEAAYQDIGCAQQABhAMgYIBRAAGEAyBggGEAAYQDIGCAcQABhAMgYICBAAGEDSAQkxMDIwNGowajmoAgiwAgE&FORM=ANAB01&PC=U531


In [5]:
# import os
# import time
# from operator import itemgetter

# import asyncio
# # import selenium
# from selenium import webdriver
# from selenium.webdriver.support.ui import WebDriverWait
# from selenium.webdriver.support import expected_conditions as EC
# from selenium.webdriver.common.by import By
# from selenium.webdriver.chrome.service import Service as ChromeService

# from concurrent.futures import ThreadPoolExecutor

In [1]:
# class Search():
#     def __init__(self, role, loc, page_depth, page_type): #split non common attributes into distinct classes (search vs ad details)
#         self.role = role
#         self.loc = loc
#         self.page_depth = page_depth
#         self.page_type = page_type
        
#         self.urls = []
#         self.drivers = []
#         self.job_search_results = {}
#         self.db_dir = f"{os.getcwd()}/db/seek_app.sqlite"
#         self.ads_in_db = None
#         # self.get_ads_in_db()
#         self.ads_not_in_db = None
#         self.SQL_command = None
#         self.SQL_command_history = []
        

        
#     def __str__(self):
#         return f"driver:{self.driver}\nRole:{self.role}\nLocation:{self.loc}\nurls:{self.urls}\nresults:{self.results}\n\n{self.job_ads}\n{self.ads_in_db}"

    
#     def generate_urls(self):
#         postcode = {
#             'Melbourne': {'state': 'VIC', 'postcode': 3000}
#             , 'Sydney': {'state': 'NSW', 'postcode': 2000}
#         }
#         search_location_string = '-'.join(str(x) for x in itemgetter('state', 'postcode')(postcode.get(self.loc)))
#         url = f'https://www.seek.com.au/{self.role.replace(" ", "-")}-jobs/in-{self.loc}-{search_location_string}' # concat only if user sepcifies a location
#         urls = [f'{url}?page={x+1}' for x in range(self.page_depth)]
#         self.urls = urls
        
    
#     def get_selenium_driver(self):
#         print('get_selenium_driver()')
#         CHROMEDRIVER_PATH=f'{os.getcwd()}/chromedriver-win64/chromedriver'
#         try:
#             options = webdriver.ChromeOptions()
#             service = ChromeService()
#             driver = webdriver.Chrome(service=service, options=options)
#         except Exception as e:
#             print('other error', e)
#         return driver


#     # Async wrapper to run the blocking Selenium code
#     async def async_scrape(self, url, executor):
#         loop = asyncio.get_running_loop()
#         return await loop.run_in_executor(executor, self.scrape_with_selenium, url)
        
#     def scrape_with_selenium(self, url):
#         try:
#             driver = self.get_selenium_driver()
#             self.drivers.append(driver)
#             driver.get(url)
#             article = driver.find_elements(By.TAG_NAME, "article")
#             with ThreadPoolExecutor(max_workers=30) as executor:
#                 executor.map(self.populate_results, article)
#         except Exception as e:
#             print('other error', e)
#         finally:
#             driver.quit()

#     def populate_results(self, article):
#         try:
#             job_ad_id = article.get_attribute("data-job-id")
#             self.job_search_results[job_ad_id] = {}
#             elements = article.find_elements(By.CSS_SELECTOR, '[data-automation]')
#             for el in elements:
#                 self.job_search_results[job_ad_id][el.get_attribute("data-automation").replace('-','_')] = el.text
#         except Exception as e:
#             print('other error', e)

#     # Main async function to scrape multiple URLs
#     async def main(self, urls):
#         results = []
#         with ThreadPoolExecutor(max_workers=3000) as executor:
#             tasks = [self.async_scrape(url, executor) for url in urls]
#             results = await asyncio.gather(*tasks)
#         print(f"\nScraped {len(results)} pages.")

#     def run_main(self):
#         # Example usage
#         if __name__ == "__main__":
#             self.generate_urls()
#             print(self.urls)
            
#             try:
#                 # Works in notebooks or async environments
#                 loop = asyncio.get_running_loop()
#             except RuntimeError:
#                 # No loop is running — safe to use asyncio.run()
#                 print('exception')
#                 asyncio.run(self.main(self.urls), debug=True)
#             else:
#                 # Loop is running — safe way to run async code inside it
#                 print('using nest method')
#                 import nest_asyncio
#                 nest_asyncio.apply()
#                 loop.create_task(self.main(self.urls))
                


In [2]:
# s = Search('Data Engineer', 'Melbourne', 10, 'search')

In [3]:
# s.run_main()

In [4]:
# @decorator_timer
# def run_search(obj):
#     obj.run_main()
# # run_search(s)

# result, exec_time = run_search(s)
# print(exec_time) # prints after 0.07347989082336426 seconds
# print(result) # takes ~3 seconds

In [78]:
def decorator_timer(some_function):
    from time import time

    def wrapper(*args, **kwargs):
        t1 = time()
        result = some_function(*args, **kwargs)
        end = time()-t1
        return result, end
    return wrapper

In [85]:
# import asyncio
# import aiohttp
# import time
# import yarl

# urls = [
#     yarl.URL(f'https://www.seek.com.au/Data-Engineer-jobs/in-Melbourne-VIC-3000?page={x+1}', encoded=True) for x in range(10)
# ]
# print(urls)
# # urls = [
# #     f'https://www.seek.com.au/Data-Engineer-jobs/in-Melbourne-VIC-3000?page={x+1}' for x in range(10)
# # ]
# urls = ['https://www.seek.com.au/Data-Engineer-jobs/in-Melbourne-VIC-3000?page=1']

# async def fetch(session, url):
#     headers = {
#         "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
#                       "AppleWebKit/537.36 (KHTML, like Gecko) "
#                       "Chrome/113.0.0.0 Safari/537.36",
#         "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8",
#         "Accept-Language": "en-US,en;q=0.5",
#         "Accept-Encoding": "gzip, deflate",
#         "Connection": "keep-alive",
#         "Upgrade-Insecure-Requests": "1",
#     }
#     async with session.get(url, headers=headers) as response:
#         text = await response.text()
#         print(f"Fetched {url} with status {response.status}")
#         return text


# async def fetch_all(urls):
#     async with aiohttp.ClientSession() as session:
#         tasks = [fetch(session, url) for url in urls]
#         return await asyncio.gather(*tasks)


# # Run this safely in a notebook or script
# async def main():
#     start_time = time.time()
#     results = await fetch_all(urls)
#     # print(results)
#     print(f"\nFetched {len(results)} pages.")
#     print('run time = ', time.time() - start_time)


# # Check if there's an existing event loop (Jupyter/IPython)
# if __name__ == "__main__":
#     try:
#         # Works in notebooks or async environments
#         loop = asyncio.get_running_loop()
#     except RuntimeError:
#         # No loop is running — safe to use asyncio.run()
#         asyncio.run(main())
#     else:
#         # Loop is running — safe way to run async code inside it
#         import nest_asyncio
#         nest_asyncio.apply()
#         loop.create_task(main())


[URL('https://www.seek.com.au/Data-Engineer-jobs/in-Melbourne-VIC-3000?page=1'), URL('https://www.seek.com.au/Data-Engineer-jobs/in-Melbourne-VIC-3000?page=2'), URL('https://www.seek.com.au/Data-Engineer-jobs/in-Melbourne-VIC-3000?page=3'), URL('https://www.seek.com.au/Data-Engineer-jobs/in-Melbourne-VIC-3000?page=4'), URL('https://www.seek.com.au/Data-Engineer-jobs/in-Melbourne-VIC-3000?page=5'), URL('https://www.seek.com.au/Data-Engineer-jobs/in-Melbourne-VIC-3000?page=6'), URL('https://www.seek.com.au/Data-Engineer-jobs/in-Melbourne-VIC-3000?page=7'), URL('https://www.seek.com.au/Data-Engineer-jobs/in-Melbourne-VIC-3000?page=8'), URL('https://www.seek.com.au/Data-Engineer-jobs/in-Melbourne-VIC-3000?page=9'), URL('https://www.seek.com.au/Data-Engineer-jobs/in-Melbourne-VIC-3000?page=10')]
Fetched https://www.seek.com.au/Data-Engineer-jobs/in-Melbourne-VIC-3000?page=1 with status 403

Fetched 1 pages.
run time =  0.05415487289428711
